In [35]:
import numpy as np
import pandas as pd
from sklearn.metrics import fbeta_score, precision_recall_curve, auc, classification_report, roc_curve
from modeling_helper import create_xy
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [3]:
flights = pd.read_csv('/Users/user/Documents/Github/flightstatus/data/dev/flights_processed.csv')

In [4]:
flights.head()

,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,...,LATITUDE_DIF,AIRLINE_CAT,TAIL_NUMBER_CAT,ORIGIN_AIRPORT_CAT,DESTINATION_AIRPORT_CAT,CITY_ORIGIN_CAT,STATE_ORIGIN_CAT,CITY_DESTINATION_CAT,STATE_DESTINATION_CAT,HOLIDAY_CAT
0,1,1,4,98,5,205.0,1448,430,61.17432,-149.99619,...,13.72534,2,1623,18,278,13,1,267,51,8
1,1,1,4,2336,10,280.0,2330,750,33.94254,-118.40807,...,7.25938,1,1557,177,236,174,6,296,10,8
2,1,1,4,840,20,286.0,2296,806,37.61900,-122.37484,...,2.40499,12,422,279,67,256,6,54,29,8
3,1,1,4,258,20,285.0,2342,805,33.94254,-118.40807,...,8.14929,1,1517,177,204,174,6,188,10,8
4,1,1,4,135,25,235.0,1448,320,47.44898,-122.30931,...,-13.72534,2,2132,278,18,267,51,13,1,8


In [6]:
datatrainval, datatest= train_test_split(flights, stratify = flights['TARGET'], test_size=0.25, random_state=12)
datatrain, dataval = train_test_split(datatrainval, stratify = datatrainval['TARGET'], test_size=0.3, random_state=12)

In [7]:
X_train, y_train = create_xy(datatrain)
X_val, y_val = create_xy(dataval)
X_test, y_test = create_xy(datatest)

In [8]:
y_train.value_counts()

0    1836715
1     963061
Name: TARGET, dtype: int64

### Dummy model

In [48]:
model = DummyClassifier(strategy='stratified')

In [49]:
model.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=None, strategy='stratified')

In [50]:
y_train_pred = model.predict_proba(X_train)[:, 1]
y_val_pred = model.predict_proba(X_val)[:, 1]
y_test_pred = model.predict_proba(X_test)[:, 1]

In [51]:
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
auc(fpr, tpr)

0.4999936279712559

In [52]:
precision, recall, thresholds = precision_recall_curve(y_test, y_test_pred)
auc(recall, precision)

0.45653241381613086

### Baseline model
The baseline model predicts that all flights are on-time.

In [32]:
#ROC
fpr, tpr, thresholds = roc_curve(y_test, np.zeros(y_test.shape[0]))
auc(fpr, tpr)

0.5

In [26]:
#PR curve
precision, recall, thresholds = precision_recall_curve(y_test, np.zeros(y_test.shape[0]))
auc(recall, precision)

0.6719890911300176